In [ ]:
# 좌표계변환 라이브러리
from pyproj import Transformer

# 데이터 분석 라이브러리
import pandas as pd
import numpy as np
import math

# 데이터 읽기
df = pd.read_csv('final_include_address1.csv', encoding='cp949')

# 유적지 위도, 경도 좌표계 변경 WGS84(EPSG:4326) -> GRS80(EPSG:5186)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:5186")
coord = transformer.transform(df['LC_LA'], df['LC_LO'])


historic_site_elevation = []

# 전체 유적지 순회
for i in range(len(coord[0])):
    
    # 유적지 도/광역시 및 시/군/구 예외처리
    if df['CTPRVN_NM'][i] == '세종특별자치시':
        si_name = '세종특별자치시'
    else:
        si_name = df['SIGNGU_NM'][i] if (df['SIGNGU_NM'][i][-1]=='구' and df['CTPRVN_NM'][i][-1] == '도') else df['CTPRVN_NM'][i] + ' ' + df['SIGNGU_NM'][i]
    
    # 유적지 주소로 해발고도 데이터 행정구역에 맞춰 읽어오기
    f=open('C:/Users/staro/Desktop/GIS/2014 '+df['CTPRVN_NM'][i]+'[ascii]/'+si_name+'.txt',"r",encoding = 'ascii')

    coords = []
    lat_list = []
    lon_list = []
    elv_list = []
    # 해당 행정구역 공간정보 모두 읽어오기
    while True:
        line = f.readline()
        if line == '' :
            break
        tmp = line.split()
        lat_list.append(tmp[0])
        lon_list.append(tmp[1])
        elv_list.append(tmp[2])

    f.close()
    
    x_diff = []
    y_diff = []
    xy_diff = []
    # 공간정보의 모든 좌표와 변환된 유적지 좌표의 유클리드 거리 계산
    for j in range(len(lat_list)):
        x_difer = abs(float(lat_list[j]) - float(coord[1][i]))
        y_difer = abs(float(lon_list[j]) - float(coord[0][i]))
        xy_diff.append(math.sqrt(x_difer*y_difer))
    
    # 가장 가까운 좌표의 해발고도 값 저장
    elev = elv_list[np.array(xy_diff).argmin()]
    historic_site_elevation.append(elev)
    print(si_name + ' ' + df['POI_NM'][i] + ' Elevation = ' + str(elev))
    
print('TASK FINISH')

In [ ]:
from pyproj import Transformer
from geopy.geocoders import Nominatim



x = 338694.53
y = 582168.12

transformer = Transformer.from_crs("EPSG:5182", "EPSG:5181")
coord = transformer.transform(y, x)
transformer = Transformer.from_crs("EPSG:5181", "EPSG:4326")
coord = transformer.transform(coord[0], coord[1])
address = geocoding_reverse(coord)
print(coord)
print(address)

In [ ]:
from pyproj import Transformer
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import folium
from folium import plugins

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

#f=open('C:/Users/staro/Downloads/제주특별자치도/2014 제주특별자치도[ascii]/제주특별자치도 제주시.txt',"rt",encoding = 'ascii')
f=open('C:/Users/staro/Desktop/GIS/2014 경상북도[ascii]/포항시 남구.txt',"r",encoding = 'ascii')

transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326")

coords = []
lat_list = []
lon_list = []
while True:
    line = f.readline()
    if line == '' :
        break
    tmp = line.split()
    coord = transformer.transform(tmp[1], tmp[0])
    coords.append(coord)
    lat_list.append(coord[0])
    lon_list.append(coord[1])
    
f.close()

dot = pd.DataFrame({'lat':lat_list, 'lon':lon_list})

# base lat, lon
centerX, centerY = dot['lat'].mean(), dot['lon'].mean()

print(centerX, centerY)

# Draw a basemap
m = folium.Map(location=[centerX, centerY], tiles='openstreetmap', zoom_start=7)

for i in range(20000,30000):
    folium.Circle(coords[i],radius = 5).add_to(m)

folium.GeoJson(yi4326 ,name='json_data').add_to(m)
    
m

In [ ]:
import geopandas as gpd
EMD = gpd.read_file('C:/Users/staro/Desktop/GIS/SIG_202302', encoding='cp949')
EMD.crs = "epsg:5179"
EMD.head()

# 용인시의 행정구역 코드는 4146
yi = EMD[EMD['SIG_CD'].str.startswith('4711')]
yi.head()



In [ ]:
# WGS 84의 좌표계로 변환하여 위도경도 데이터를 생성
yi4326 = yi.to_crs(epsg=4326)
yi4326 = yi4326.sort_values('SIG_KOR_NM')
display(yi4326.head(),yi4326.shape)

print(type(yi4326))

In [ ]:
import json
import folium
from folium import Choropleth
import pandas as pd

dosi = pd.read_csv('do_si.csv', encoding='cp949')

dosi2 = dosi.values.tolist()

with open('korea_geojson2.geojson', encoding='UTF-8') as f:
    korea_geo = json.load(f)
    
for x in korea_geo['features']:
    x['id'] = x['properties']['CTP_KOR_NM'] 
    
#for idx, _ in enumerate(korea_geo['features']):
#    print(korea_geo['features'][idx]['id'])
    
m = folium.Map(
    location=[35.559819, 126.963895],
    zoom_start=6, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    korea_geo,
    name='지역구'
).add_to(m)

m.choropleth(geo_data=korea_geo,
             data=dosi2, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='id',
             legend_name="지역구별 유적지수"
            )
m